In [2]:
import warnings
warnings.filterwarnings('ignore')
import glob
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder
import random
import os
from sklearn.metrics import log_loss

In [80]:
d = "C:\kaggle_data\credit_card"
lst = os.listdir(d)
print(lst)
train = pd.read_csv(d + '\\' +lst[3])
test = pd.read_csv(d + '\\' +lst[2])
ss = pd.read_csv(d + '\\' +lst[1])
train = train.drop(['index'], axis=1)
train.fillna('NAN', inplace=True) 
test = test.drop(['index'], axis=1)
test.fillna('NAN', inplace=True)

['.ipynb_checkpoints', 'sample_submission.csv', 'test.csv', 'train.csv', 'Untitled.ipynb']


# 전처리

### family_type
- 'Married', 'Civil marriage'
- 'Separated',  'Widow'
- 'Single / not married'

In [81]:
# Married, Civil marriage
train['income_per_size'] = np.log(train['income_total']/train['family_size'])
test['income_per_size'] = np.log(test['income_total']/test['family_size'])
train.loc[(train['family_type']=='Married')|(train['family_type']=='Civil marriage'),'income_per_size']\
= train['income_per_size'] * 2

test.loc[(test['family_type']=='Married')|(test['family_type']=='Civil marriage'),'income_per_size']\
= test['income_per_size'] * 2

In [82]:
def simple_marry(x):
    if x == 'Married' or x =='Civil marriage':
        return '0'
    elif x == 'Separated' or x == 'Widow':
        return '1'
    else:
        return '2'

In [83]:
# 안 덮어쓰기
for df in [train,test]:
    df['family_bins'] = df['family_type'].apply(simple_marry)

In [84]:
# income_total을 로그변환
train['income_total'] = np.log(train['income_total'])
test['income_total'] = np.log(test['income_total'])

In [85]:
# car와 reality를 합친 새로운 칼럼 careality
train['car'] =train['car'].apply(lambda x: int(x=='Y'))
train['reality'] =train['reality'].apply(lambda x: int(x=='Y'))
test['car'] =test['car'].apply(lambda x: int(x=='Y'))
test['reality'] =test['reality'].apply(lambda x: int(x=='Y'))

train['careality'] = train['car'] + train['reality']
train = train.drop(['car', 'reality'],1)

test['careality'] = test['car'] + test['reality']
test = test.drop(['car', 'reality'],1)

In [86]:
object_col = []
for col in train.columns:
    if train[col].dtype == 'object':
        object_col.append(col)

In [87]:
enc = OneHotEncoder()
enc.fit(train.loc[:,object_col])


train_onehot_df = pd.DataFrame(enc.transform(train.loc[:,object_col]).toarray(), 
             columns=enc.get_feature_names(object_col))
train.drop(object_col, axis=1, inplace=True)
train = pd.concat([train, train_onehot_df], axis=1)

In [88]:
test_onehot_df = pd.DataFrame(enc.transform(test.loc[:,object_col]).toarray(), 
             columns=enc.get_feature_names(object_col))
test.drop(object_col, axis=1, inplace=True)
test = pd.concat([test, test_onehot_df], axis=1)

# 
- income_per_size 정규화
- feature 추가(binary)
- 두개 다

In [89]:
train['income_per_size']

0        23.050696
1        22.641107
2        24.647711
3        23.050696
4        22.548067
           ...    
26452    21.875123
26453    11.407565
26454    23.786146
26455    12.049419
26456    21.218115
Name: income_per_size, Length: 26457, dtype: float64

In [90]:
# # 15.75
# # -0.92


# 제곱
for df in [train,test]:
    df['income_per_size'] = df['income_per_size'].apply(lambda x: x**2)
    
    

c = 'income_per_size'
mean = train[c].mean()
std = train[c].std()
# 기존 
train[c] = (train[c]-mean)/std
test[c] = (test[c]-mean)/std

In [91]:
# train['ips_bin'].value_counts()

In [92]:
#####################
c = 'income_total'
k = 2.2
mean = train[c].mean()
std = train[c].std()
idxs = train.loc[(train[c]>= mean + k*std)|\
                (train[c]<= mean - k*std)].index
train = train.drop(idxs).reset_index(drop=True)

# 기존 
train[c] = (train[c]-mean)/std
test[c] = (test[c]-mean)/std
# # 새로운
# mean = train[c].mean()
# std = train[c].std()
# train[c] = (train[c]-mean)/std
# test[c] = (test[c]-mean)/std

In [93]:
#####################
c = 'begin_month'
k = 2.2
mean = train[c].mean()
std = train[c].std()
idxs = train.loc[(train[c]>= mean + k*std)|\
                (train[c]<= mean - k*std)].index
train = train.drop(idxs).reset_index(drop=True)

# 기존 
train[c] = (train[c]-mean)/std
test[c] = (test[c]-mean)/std
# # 새로운
# mean = train[c].mean()
# std = train[c].std()
# train[c] = (train[c]-mean)/std
# test[c] = (test[c]-mean)/std

In [94]:
######################
c = 'DAYS_BIRTH'
k = 2.2
mean = train[c].mean()
std = train[c].std()
idxs = train.loc[(train[c]>= mean + k*std)|\
                (train[c]<= mean - k*std)].index
train = train.drop(idxs).reset_index(drop=True)

#기존 
train[c] = (train[c]-mean)/std
test[c] = (test[c]-mean)/std
# ## 새로운
# mean = train[c].mean()
# std = train[c].std()
# train[c] = (train[c]-mean)/std
# test[c] = (test[c]-mean)/std

In [95]:
# out: 아웃라이어
out_train = train.loc[train.DAYS_EMPLOYED>0]
in_train = train.loc[train.DAYS_EMPLOYED<=0]


out_test = test.loc[test.DAYS_EMPLOYED>0]
in_test = test.loc[test.DAYS_EMPLOYED<=0]

<!-- ###
- DAYS_EMPLOYED -->

# 모델링, 학습

In [96]:
out_train = out_train.reset_index()
in_train = in_train.reset_index()

out_test = out_test.reset_index()
in_test = in_test.reset_index()

In [97]:
idx_out_train = out_train.pop('index')
idx_out_test = out_test.pop('index')

idx_in_train = in_train.pop('index')
idx_in_test = in_test.pop('index')

In [98]:
###################
c = 'DAYS_EMPLOYED'
k = 2.2
mean = in_train[c].mean()
std = in_train[c].std()
idxs = in_train.loc[(in_train[c]>= mean + k*std)|\
                (in_train[c]<= mean - k*std)].index
in_train = in_train.drop(idxs).reset_index(drop=True)

# 기존 
train[c] = (train[c]-mean)/std
test[c] = (test[c]-mean)/std
# # 새로운
# mean = train[c].mean()
# std = train[c].std()
# train[c] = (train[c]-mean)/std
# test[c] = (test[c]-mean)/std

In [99]:
len(train.columns)

59

In [100]:
out_train.drop('DAYS_EMPLOYED',inplace=True,axis=1)
out_test.drop('DAYS_EMPLOYED',inplace=True,axis=1)

In [101]:
out_train.head()

,child_num,income_total,DAYS_BIRTH,FLAG_MOBIL,work_phone,phone,email,family_size,begin_month,credit,...,occyp_type_Private service staff,occyp_type_Realty agents,occyp_type_Sales staff,occyp_type_Secretaries,occyp_type_Security staff,occyp_type_Waiters/barmen staff,family_bins_0,family_bins_1,family_bins_2,ips_bin
0,0,-0.114150,-1.670540,1,0,1,0,1.0,-0.896630,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
1,0,0.542746,-1.716051,1,0,0,0,2.0,-0.655166,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1
2,0,-0.114150,-1.272859,1,0,1,0,2.0,1.155818,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1
3,0,0.408522,-1.536868,1,0,1,0,1.0,1.276550,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
4,0,-0.434746,-1.236403,1,0,1,0,1.0,0.129594,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0


In [102]:
def tr(train):
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    folds=[]
    losses=[]
    for train_idx, valid_idx in skf.split(train, train['credit']):
        folds.append((train_idx, valid_idx))
    random.seed(42)
    lgb_models={}
    for fold in range(5):
        print(f'===================================={fold+1}============================================')
        train_idx, valid_idx = folds[fold]
        X_train, X_valid, y_train, y_valid = train.drop(['credit'],axis=1).iloc[train_idx].values, train.drop(['credit'],axis=1).iloc[valid_idx].values,\
                                             train['credit'][train_idx].values, train['credit'][valid_idx].values 
        lgb = LGBMClassifier(n_estimators=1000)
        lgb.fit(X_train, y_train, 
                eval_set=[(X_train, y_train), (X_valid, y_valid)], 
                early_stopping_rounds=30,
               verbose=100)
        lgb_models[fold]=lgb
        losses.append(log_loss(y_valid, lgb.predict_proba(X_valid)))
        print(f'================================================================================\n\n')
    ls = sum(losses)/5
    print(ls)
    return ls, lgb_models

In [103]:
out_loss, out_models = tr(out_train)

====================================1============================================
Training until validation scores don't improve for 30 rounds
[100]	training's multi_logloss: 0.405894	valid_1's multi_logloss: 0.728349
Early stopping, best iteration is:
[86]	training's multi_logloss: 0.433012	valid_1's multi_logloss: 0.724757


====================================2============================================
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[45]	training's multi_logloss: 0.535478	valid_1's multi_logloss: 0.748793


====================================3============================================
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[46]	training's multi_logloss: 0.53242	valid_1's multi_logloss: 0.740544


====================================4============================================
Training until validation scores don't improve for 30 rounds
Early stopping, best 

In [104]:
in_loss, in_models = tr(in_train)

====================================1============================================
Training until validation scores don't improve for 30 rounds
[100]	training's multi_logloss: 0.609358	valid_1's multi_logloss: 0.757097
[200]	training's multi_logloss: 0.504673	valid_1's multi_logloss: 0.748089
Early stopping, best iteration is:
[207]	training's multi_logloss: 0.498296	valid_1's multi_logloss: 0.747412


====================================2============================================
Training until validation scores don't improve for 30 rounds
[100]	training's multi_logloss: 0.611482	valid_1's multi_logloss: 0.745917
[200]	training's multi_logloss: 0.510918	valid_1's multi_logloss: 0.733993
Early stopping, best iteration is:
[236]	training's multi_logloss: 0.483029	valid_1's multi_logloss: 0.732723


====================================3============================================
Training until validation scores don't improve for 30 rounds
[100]	training's multi_logloss: 0.613942	valid_

In [ ]:
## ips 제곱
0.7398746001125598
0.731703572697814



## 제곱 없이 15.75빼고 표편으로 나눠줌
0.739873725695487
0.732629847481416


## 제곱, 표준화, binary
0.7397848695705913
0.7316433141400384

# binary만
0.7452942657876084
0.7339222178367916

In [570]:
### 기존의 평균 표준편차 사용 정규화
# 0.7398746001125598 
# 0.731703572697814  

# binary feature
# 0.7447143745928957 0.005 증가
# 0.7332622413020282 0.002 증가

# 정규화
# 0.7398098658534071 0.00007 감소
# 0.7318043493395331 0.0001 증가

# 둘 다

# 0.7398098658534071 0.00007 감소
# 0.7318043493395331 0.0001 증가


### ips 제곱하고 정규화
# 0.739873725695487  0.00000~ 감소
# 0.7316585777211151 0.0001 감소


####  마지막
0.739873725695487
0.7316585777211151




# 0.7447143745928957 0.005 증가
# 0.7332622413020282 0.002 증가

0.7316585777211151

In [571]:
print(in_loss)

0.732629847481416


In [105]:
in_zeros = np.zeros([len(in_test),3])
for fold in range(5):
    in_zeros += in_models[fold].predict_proba(in_test)/5
in_output = pd.DataFrame(in_zeros)
in_output = in_output.reindex(index=pd.Index(idx_in_test))

In [106]:
out_zeros = np.zeros([len(out_test),3])
for fold in range(5):
    out_zeros += out_models[fold].predict_proba(out_test)/5
out_output = pd.DataFrame(out_zeros)
out_output = out_output.reindex(index=pd.Index(idx_out_test))

In [107]:
in2 = np.concatenate((in_zeros,np.array(idx_in_test).reshape(-1,1)),axis=1)
out2 = np.concatenate((out_zeros,np.array(idx_out_test).reshape(-1,1)),axis=1)

In [108]:
output = pd.DataFrame(np.concatenate((in2,out2),axis=0),columns=[0,1,2,'index'])

In [109]:
output['index'] = output['index'].astype('int')

In [110]:
ss = pd.read_csv(d + '\\' +lst[1])

In [111]:
for i in range(len(output)):
    row = output.loc[output['index']==i]
    ss.iloc[i,1:] = row.iloc[:,:3]

In [112]:
output

,0,1,2,index
0,0.083646,0.177263,0.739091,1
1,0.031386,0.091128,0.877486,2
2,0.101208,0.133666,0.765126,3
3,0.071951,0.146165,0.781885,4
4,0.059793,0.130476,0.809731,5
...,...,...,...,...
9995,0.040524,0.067478,0.891998,9968
9996,0.104505,0.089859,0.805636,9981
9997,0.135468,0.191587,0.672945,9988
9998,0.214337,0.158605,0.627058,9992


In [113]:
ss

,index,0,1,2
0,26457,0.041973,0.140227,0.817800
1,26458,0.083646,0.177263,0.739091
2,26459,0.031386,0.091128,0.877486
3,26460,0.101208,0.133666,0.765126
4,26461,0.071951,0.146165,0.781885
...,...,...,...,...
9995,36452,0.106584,0.245990,0.647426
9996,36453,0.179093,0.287460,0.533446
9997,36454,0.176063,0.084956,0.738981
9998,36455,0.075697,0.202040,0.722262


In [115]:
ss.to_csv('sq_no_bin.csv', index=False)

In [205]:
output.head()

,0,1,2,index
0,0.254328,0.176459,0.569213,1
1,0.037702,0.077119,0.885179,2
2,0.150319,0.105418,0.744263,3
3,0.074441,0.168475,0.757084,4
4,0.057947,0.112467,0.829586,5


In [206]:
output.tail()

,0,1,2,index
9995,0.045256,0.062820,0.891924,9968
9996,0.113289,0.091170,0.795541,9981
9997,0.162660,0.185026,0.652315,9988
9998,0.231756,0.133885,0.634359,9992
9999,0.083838,0.910228,0.005934,9994
